In [ ]:
import torch
import torch.nn as nn
import numpy as np

from torch.utils.data import BatchSampler, SequentialSampler

from DeepCCAModels import DeepCCA

from torch.utils.data import DataLoader, Dataset

from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score

import random

from models import total, ConvNet, train_with_reg_cv, Conv_CCAA, Regressor

from objectives import cca_loss

from sklearn.metrics import mean_absolute_error

In [ ]:
device = torch.device('cuda')
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
torch.manual_seed(111)

In [ ]:
params = {}
params['cov_out_dim'] = [30,40,60]
params['cov_cca_dim'] = [20,40,30]
params['batch_size'] = [400,500]
params['hidden_dim'] = [256,512,768]
params['epoch_num_1'] = [100]
params['learning_rate'] = [1e-4, 3e-4]

In [ ]:
batch_size = random.choice(params['batch_size'])
cov_out_dim = random.choice(params['cov_out_dim'])
cov_cca_dim = random.choice(params['cov_cca_dim'])
learning_rate_1 = random.choice(params['learning_rate'])
hidden_dim = random.choice(params['hidden_dim'])
epoch_num_1 = random.choice(params['epoch_num_1'])

current_setting = (batch_size, 
                   cov_out_dim, 
                   hidden_dim, cov_cca_dim, 
                   learning_rate_1, epoch_num_1)

if current_setting in seen_settings:
    continue
else:
    seen_settings.add(current_setting)
print (current_setting)

train_ite = DataLoader(mosi_train, batch_size = batch_size, shuffle = True)
train_ite_1 = DataLoader(mosi_train, batch_size = 1, shuffle = False)
test_ite = DataLoader(mosi_test, batch_size = 1, shuffle = False)
val_ite = DataLoader(mosi_val, batch_size = 1, shuffle = False)

cca_loss_1 = cca_loss(cov_cca_dim, False, device).loss
con_cca = Conv_CCAA(cov_out_dim).cuda()

optimizer_1 = torch.optim.RMSprop(con_cca.parameters(), lr=learning_rate_1)

for epoch in range(epoch_num_1):
    con_cca.train()
    for i1, train_batch in enumerate(train_ite):
        con_cca.zero_grad()

        atmat1 = train_batch[0].cuda()

        vtmat1 = train_batch[1].cuda()

        out1, out2 = con_cca(atmat1, vtmat1)

        loss1 = cca_loss_1(out1, out2)
        loss1.backward()
        optimizer_1.step()

train_out = []
test_out = []
val_out = []
with torch.no_grad():
    for o1, batch1 in enumerate(train_ite_1):
        con_cca.eval()
        t11, t12 = con_cca(batch1[0].cuda(), batch1[1].cuda())
        train_out.append(torch.cat((t11, t12), dim = 1).cpu().detach().numpy().reshape(-1))

    for o2, batch2 in enumerate(test_ite):
        con_cca.eval()
        t21, t22 = con_cca(batch2[0].cuda(), batch2[1].cuda())
        test_out.append(torch.cat((t21,t22), dim=1).cpu().detach().numpy().reshape(-1))
    for o3, batch3 in enumerate(val_ite):
        con_cca.eval()
        t31, t32 = con_cca(batch3[0].cuda(), batch3[1].cuda())
        val_out.append(torch.cat((t31, t32),dim=1).cpu().detach().numpy().reshape(-1))

final_train = np.concatenate((np.array(train_out),mosi_b_train), axis = 1 )

final_test = np.concatenate((np.array(test_out),mosi_b_test), axis = 1 )

final_val = np.concatenate((np.array(val_out), mosi_b_val), axis = 1 )


np.save('new_train', final_train)
np.save('new_test', final_test)
np.save('new_val', final_val)